#####

In [22]:
%%time
#✅ version: 05--- developing..... to solve allocation qty > open demand qty 
#✅ for Luanna maintain hotloading allocation list into HJ **********************
#✅ notice: important!!  setup:
#✅ HJ Browser screen zoom in setup = 90%;  display seting = 1920x1080;   scale setting value = 100%
#✅ created by JimShen on Feb.22.2023

import threading
import pandas as pd
import pyautogui
import pyperclip
import win32gui
import win32con
import win32com.client
import win32api
import pywintypes
import pymsgbox
import time
from pyautogui import ImageNotFoundException
import sys

pyautogui.FAILSAFE = True
pyautogui.PAUSE = 1

#✅ 以下为将浏览器窗口最大化
def get_all_hwnd(hwnd, mouse):
    if (win32gui.IsWindow(hwnd) and
        win32gui.IsWindowEnabled(hwnd) and
        win32gui.IsWindowVisible(hwnd)):
        hwnd_map.update({hwnd: win32gui.GetWindowText(hwnd)})

hwnd_map = {}
win32gui.EnumWindows(get_all_hwnd, 0)

for h, t in hwnd_map.items():
    if t :
        if t == 'Wanek_hotloading_allocation_v02 - Jupyter Notebook — Mozilla Firefox':
            # h 为想要放到最前面的窗口句柄
            # print(h,'&&',t)

            win32gui.BringWindowToTop(h)
            shell = win32com.client.Dispatch("WScript.Shell")
            shell.SendKeys('%')

            # 被其他窗口遮挡，调用后放到最前面
            win32gui.SetForegroundWindow(h)   

#✅ 确认用户是否已登录 HJ 页面
msgbox = pymsgbox.confirm(
    text="Have you already signed in Wanek HJ Item Allocation webpage?",
    title='Notice:',
    buttons=['Y', 'N']
)

if msgbox == "N":
    pymsgbox.alert('Please sign in Wanek HJ HotLoading Items Allocation (page2237)', 'Notice', 'OK')
    sys.exit(0)  
elif msgbox == "Y":
    pymsgbox.alert("Please switch to HJ Item Allocation screen in 5 seconds !!!!!", 'WARNING!!!!!!!!', 'OK')
    time.sleep(5)
else:
    pymsgbox.alert("You must choose 'Y' or 'N'", 'Warning', 'OK')
    sys.exit(0)

# paste()
def paste(text):
    pyperclip.copy(text)
    pyautogui.hotkey('ctrl','v')

# ✅picture capture
def wait_for_image(image_path, confidence=0.5, region=None, timeout=10):
    """在屏幕上等待某个图像出现，直到超时"""
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            pos = pyautogui.locateCenterOnScreen(
                image_path,
                confidence=confidence,
                grayscale=True,
                region=region
            )
            if pos:
                print(f"✅ Found {image_path} at: {pos}")
                return pos
        except ImageNotFoundException:
            pass
        time.sleep(0.5)
    raise Exception(f"⛔ Timeout: {image_path} not found within {timeout} seconds.")

# ✅不等待、只查找一次 的图片检测函数
def quick_find_image(image_path, confidence=0.7, region=None):
    try:
        pos = pyautogui.locateCenterOnScreen(image_path, confidence=confidence, grayscale=True, region=region)
        if pos:
            print(f"✅ Found {image_path} at: {pos}")
            return pos
        else:
            return None
    except Exception:
        return None


#✅ import excel file Wanek_Allocation_List.xlsx into df that will be entered into excel:
df = pd.read_excel(r'C:\Users\jishen\Downloads\Wanek_Allocation_List.xlsx',sheet_name=0,skiprows=1,
                   dtype={'Item':str,'Priority':str,'WHs':str,'Total':str,'Destination':str})
df['Due'] = pd.to_datetime(df['Due'],errors='coerce')
df['Due'] = df['Due'].dt.strftime('%m/%d/%Y')


# ✅ import excel file Allocated_List.xlsx into df2 then loaded into dictionary:
df2 = pd.read_excel(r'C:\Users\jishen\Downloads\Allocated_List.xlsx',sheet_name=0,usecols='B,D',dtype={'Production Item':str})
df2['*MFG Date'] = pd.to_datetime(df2['*MFG Date'],errors='coerce')
df2['*MFG Date'] = df2['*MFG Date'].dt.strftime('%m/%d/%Y')
df2 = df2[df2['*MFG Date']==df['Due'][0]]
df2.reset_index(inplace=True)
df2.drop(columns='index',inplace=True)

# ✅ 
d={}    # Allocated_List items
d2={}   # 是程序运行过程中构造的一个 “已处理物料编号”集合（字典）。用于防止重复处理相同的物料。

for x in range(df2.shape[0]):
    d[df2['Production Item'][x]] = ''

# ✅  main: go through df each item number ************************************************************************    
for i in range(df.shape[0]):
    # if the item number in allocated list then go to next i
    if df['Item'][i] in d or df['Item'][i] in d2:
        continue   # 不执行以下，go to next i
    
    # if item in d:
    if df['Item'][i] not in d:      # 只有“没在 allocated list 中的 item”才会走到这里, 如果存在于allocated list中
        # click refresh button 
        # while True:
        #     refresh_pos = wait_for_image('refresh.png',confidence=0.75,
        #                                                       region=(1300,169,1833,272))
        #     if refresh_pos:
        #         break
        # pyautogui.moveTo([refresh_pos[0],refresh_pos[1]])
        # pyautogui.click()
        # time.sleep(0.15)             

        # click Add Item Allocation button
        while True:
            additems_pos = wait_for_image('additems.png',confidence=0.8,
                                                              region=(1300,169,1833,272))
            if additems_pos:
                break
        pyautogui.moveTo([additems_pos[0],additems_pos[1]])
        pyautogui.click(clicks=1)
        time.sleep(0.25)  
        
        # enter item number 
        # pyautogui.moveTo([300,374])
        # pyautogui.click()
        # paste(df['Item'][i])

        while True:
            porducitonitem_pos = wait_for_image('productionitem.png',confidence=0.80,
                                                              region=(60,280,500,700))
            if porducitonitem_pos:
                break
        pyautogui.moveTo([porducitonitem_pos[0],porducitonitem_pos[1]+30])
        pyautogui.click()
        paste(df['Item'][i]) 

        # enter WHs
        # pyautogui.moveTo([300,490])
        # pyautogui.click(clicks=3)
        while True:
            outboundwarehouse_pos = wait_for_image('outboundwarehouse.png',confidence=0.75,
                                                              region=(60,280,500,700))
            if outboundwarehouse_pos:
                break
        pyautogui.moveTo([outboundwarehouse_pos[0],outboundwarehouse_pos[1]+30])
        pyautogui.click(clicks=3)

        if df['WHs'][i]=='1':
            pass
        elif df['WHs'][i]=='5':
                pyautogui.typewrite('5',0.00005)
        elif df['WHs'][i]=='15':
                pyautogui.typewrite('15',0.00005)
        elif df['WHs'][i]=='17':
                pyautogui.typewrite('17',0.00005)
        elif df['WHs'][i]=='28':
                pyautogui.typewrite('28',0.00005)
        elif df['WHs'][i]=='42':
                pyautogui.typewrite('42',0.00005)
        elif df['WHs'][i]=='335':
                pyautogui.typewrite('335',0.00005)
        elif df['WHs'][i]=='ECR':
                pyautogui.typewrite('ecr',0.00005)
        elif df['WHs'][i]=='Direct Customer':
                pyautogui.typewrite('direct',0.00005)
        elif df['WHs'][i]=='12':
                pyautogui.typewrite('12',0.00005)
        elif df['WHs'][i]=='20':
                pyautogui.typewrite('20',0.00005)                
        elif df['WHs'][i]=='49':
                pyautogui.typewrite('49',0.00005)
        elif df['WHs'][i]=='50':
                pyautogui.typewrite('50',0.00005)
        elif df['WHs'][i]=='60':
                pyautogui.typewrite('60',0.00005)
        elif df['WHs'][i]=='70':
                pyautogui.typewrite('70',0.00005)   
        elif df['WHs'][i]=='101':
                pyautogui.typewrite('101',0.00005) 
        elif df['WHs'][i]=='213':
                pyautogui.typewrite('213',0.00005) 
        elif df['WHs'][i]=='215':
                pyautogui.typewrite('215',0.00005) 
        elif df['WHs'][i]=='242':
                pyautogui.typewrite('242',0.00005) 
        else:
            pass

        # enter Allocation Qty 
        # pyautogui.moveTo([159,546])
        # pyautogui.click()
        # paste(df['Total'][i]) 

        while True:
            allocationqty_pos = wait_for_image('allocationqty.png',confidence=0.75,
                                                              region=(60,280,500,700))
            if allocationqty_pos:
                break
        pyautogui.moveTo([allocationqty_pos[0],allocationqty_pos[1]+30])
        pyautogui.click()
        paste(df['Total'][i]) 
        
        # enter Sequence 
        while True:
            sequence_pos = wait_for_image('sequence.png',confidence=0.75,
                                                              region=(60,280,500,700))
            if sequence_pos:
                break
        pyautogui.moveTo([sequence_pos[0],sequence_pos[1]+30])
        pyautogui.click(clicks=3)
        paste(df['Priority'][i]) 
        pyautogui.press('enter')

        # ✅ allocation 输入完item, whse, qty, sequence后出现的几种可能情况 
        result = None
        start_time2 = time.time()
        timeout2 = 35  # 最多等15秒，你可以按实际情况调整
        
        while True:
            if time.time() - start_time2 > timeout2:
                raise Exception("Timeout waiting for any result dialog.")
            
            # 1. 新增成功
            adsucc_pos = quick_find_image('adsucc.png', confidence=0.75, region=(100,300,1000,500))
            if adsucc_pos:
                result = 'adsucc'
                break  # 跳出 while 循环，进入后续处理。

        
            # 2. 已存在
            alreadyexist_pos = quick_find_image('alreadyexist.png', confidence=0.75, region=(100,300,1000,500))
            if alreadyexist_pos:
                result = 'alreadyexist'
                break
        
            # 3. 超出需求
            morthandeman_pos = quick_find_image('MoreThenDemand.png', confidence=0.75, region=(100,300,1000,500))
            if morthandeman_pos:
                result = 'morthandeman'
                break
        
            time.sleep(0.2)  # 每轮等待0.2秒，降低CPU负载
        
        # --- 分支操作 ---
        if result == 'adsucc':
            # 新增成功后，点击“成功”提示
            pyautogui.moveTo([adsucc_pos[0], adsucc_pos[1]])
            pyautogui.click()
            time.sleep(0.25)
            n = 0
            for j in range(df.shape[0]):
                if df['Item'][i]== df['Item'][j]:
                    n += 1
                    if n == 1:
                        continue
                    # page2238 outbound warehouse list (look for destination whse)       
                    if df['WHs'][j] == '1':
                        whs_pos = wait_for_image('1.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '101':
                        whs_pos = wait_for_image('101.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '12':
                        whs_pos = wait_for_image('12.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '15':
                        whs_pos = wait_for_image('15.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '17':
                        whs_pos = wait_for_image('17.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '20':
                        whs_pos = wait_for_image('20.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '213':
                        whs_pos = wait_for_image('213.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '215':
                        whs_pos = wait_for_image('215.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '242':
                        whs_pos = wait_for_image('242.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '28':
                        whs_pos = wait_for_image('28.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '335':
                        whs_pos = wait_for_image('335.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '42':
                        whs_pos = wait_for_image('42.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '49':
                        whs_pos = wait_for_image('49.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '5':
                        whs_pos = wait_for_image('5.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '50':
                        whs_pos = wait_for_image('50.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '60':
                        whs_pos = wait_for_image('60.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '70':
                        whs_pos = wait_for_image('70.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == 'Direct Customer':
                        whs_pos = wait_for_image('DirectCustomer.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == 'ECR':
                        whs_pos = wait_for_image('ECR.png',confidence=0.8,region=(146,380,305,959))       
                    else:
                        pass
                       
                    # click whs_pos -- allocation qty
                    if whs_pos == None:
                        cancel_pos = wait_for_image('cancelsign.png',confidence=0.8,
                                                        region=(0,0,1920,1080))
                        pyautogui.moveTo(cancel_pos)    # click "cancel" 
                        pyautogui.click() 
                        time.sleep(0.2)   
                    else:
                        pyautogui.moveTo(626,whs_pos[1])  # move to allocation qty  ---- need maintain
                        pyautogui.click(clicks=2)
                        paste(df['Total'][j])
    
                        pyautogui.press('tab')    # move to sequence   
                        paste(df['Priority'][j])    # sequence=priority
                    
                        pyautogui.moveTo([1575,204])    # item allocation click button "save"    ---- need maintain
                        pyautogui.click()          
                        time.sleep(0.2)    #  current is still in item allocation whse list screen 
    
            btlist_pos = wait_for_image('returnbacktolist.png',confidence=0.6,
                                                                  region=(0,0,1920,1080))
            pyautogui.moveTo(btlist_pos)    # click "return back to list" 
            pyautogui.click() 
            time.sleep(0.25)
            d2[df['Item'][i]] = 1    # put item number into dictionary   
            
        
            # 点击返回
            # retbk_pos = wait_for_image('retbk.png', confidence=0.8, region=(0,0,1920,1080))
            # pyautogui.moveTo([retbk_pos[0], retbk_pos[1]])
            # pyautogui.click()
            # time.sleep(0.25)
            # d2[df['Item'][i]] = 1
        
        elif result == 'alreadyexist':
            # 已存在提示，点击关闭
            pyautogui.moveTo([alreadyexist_pos[0], alreadyexist_pos[1]])
            pyautogui.click()
            time.sleep(0.15)
            # 可以加个 return 或 continue，视你的主流程而定
            for j in range(df.shape[0]):
                if df['Item'][i]== df['Item'][j]:
                    # page2238 outbound warehouse list (look for destination whse)       
                    if df['WHs'][j] == '1':
                        whs_pos = wait_for_image('1.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '101':
                        whs_pos = wait_for_image('101.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '12':
                        whs_pos = wait_for_image('12.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '15':
                        whs_pos = wait_for_image('15.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '17':
                        whs_pos = wait_for_image('17.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '20':
                        whs_pos = wait_for_image('20.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '213':
                        whs_pos = wait_for_image('213.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '215':
                        whs_pos = wait_for_image('215.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '242':
                        whs_pos = wait_for_image('242.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '28':
                        whs_pos = wait_for_image('28.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '335':
                        whs_pos = wait_for_image('335.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '42':
                        whs_pos = wait_for_image('42.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '49':
                        whs_pos = wait_for_image('49.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '5':
                        whs_pos = wait_for_image('5.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '50':
                        whs_pos = wait_for_image('50.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '60':
                        whs_pos = wait_for_image('60.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == '70':
                        whs_pos = wait_for_image('70.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == 'Direct Customer':
                        whs_pos = wait_for_image('DirectCustomer.png',confidence=0.8,region=(146,380,305,959))
                    elif df['WHs'][j] == 'ECR':
                        whs_pos = wait_for_image('ECR.png',confidence=0.8,region=(146,380,305,959))       
                    else:
                        pass
                       
                    # click whs_pos -- allocation qty
                    if whs_pos == None:
                        cancel_pos = wait_for_image('cancelsign.png',confidence=0.8,
                                                        region=(0,0,1920,1080))
                        pyautogui.moveTo(cancel_pos)    # click "cancel" 
                        pyautogui.click() 
                        time.sleep(0.2)   
                    else:
                        pyautogui.moveTo(626,whs_pos[1])  # move to allocation qty  ---- need maintain
                        pyautogui.click(clicks=2)
                        paste(df['Total'][j])
    
                        pyautogui.press('tab')    # move to sequence   
                        paste(df['Priority'][j])    # sequence=priority
                    
                        pyautogui.moveTo([1575,204])    # item allocation click button "save"    ---- need maintain
                        pyautogui.click()          
                        time.sleep(0.2)    #  current is still in item allocation whse list screen 
    
            btlist_pos = wait_for_image('returnbacktolist.png',confidence=0.6,
                                                                  region=(0,0,1920,1080))
            pyautogui.moveTo(btlist_pos)    # click "return back to list" 
            pyautogui.click() 
            time.sleep(0.25)
            d2[df['Item'][i]] = 1    # put item number into dictionary             
            
        
        elif result == 'morthandeman':
            # 超过需求提示
            pyautogui.moveTo([morthandeman_pos[0], morthandeman_pos[1]])
            pyautogui.click()
            time.sleep(0.1)
            # 回退箭头
            backarrow_pos = wait_for_image('backarrow.png', confidence=0.8, region=(0,0,1920,1080))
            pyautogui.moveTo([backarrow_pos[0], backarrow_pos[1]])
            pyautogui.click()
            time.sleep(0.1)
        else:
            print("unknow reason, please check the code！")
      
#  df3.to_excel(r'C:\Users\jishen\Downloads\WN3_AllocationOverCO.xlsx', sheet_name='OverCO')         
pyautogui.alert(text = 'total finished {} rows records, please compare your excel file and HJ data '.format(df.shape[0]),
                title='Notice',button='OK')


✅ Found additems.png at: Point(x=1495, y=205)
✅ Found productionitem.png at: Point(x=143, y=350)
✅ Found outboundwarehouse.png at: Point(x=163, y=464)
✅ Found allocationqty.png at: Point(x=145, y=521)
✅ Found sequence.png at: Point(x=129, y=577)
✅ Found adsucc.png at: Point(x=181, y=338)
✅ Found 17.png at: Point(x=200, y=529)
✅ Found returnbacktolist.png at: Point(x=1513, y=204)
✅ Found additems.png at: Point(x=1495, y=205)
✅ Found productionitem.png at: Point(x=143, y=350)
✅ Found outboundwarehouse.png at: Point(x=163, y=464)
✅ Found allocationqty.png at: Point(x=145, y=521)
✅ Found sequence.png at: Point(x=129, y=577)
✅ Found adsucc.png at: Point(x=181, y=338)
✅ Found returnbacktolist.png at: Point(x=1513, y=204)
✅ Found additems.png at: Point(x=1495, y=205)
✅ Found productionitem.png at: Point(x=143, y=350)
✅ Found outboundwarehouse.png at: Point(x=163, y=464)
✅ Found allocationqty.png at: Point(x=145, y=521)
✅ Found sequence.png at: Point(x=129, y=577)
✅ Found adsucc.png at: Point(

'OK'